In [2]:
import os,sys,re,json,logging,yaml
import pandas as pd
from pandas.io.sql import read_sql_query
import psycopg2,psycopg2.extras
import numpy as np

# Notes 12/6/21
## ToDo
* Adjust all hard-coding to functions, with drug_name input instead of specific for gabapentin
* Adjust MeSH type extraction to prioritize mesh_list when present


# 1.AACT
AACT is a publicly available relational database that contains all information (protocol and result data elements) about every study registered in ClinicalTrials.gov. Content is downloaded from ClinicalTrials.gov daily and loaded into AACT.

https://aact.ctti-clinicaltrials.org/

**Utility for project**:
* **"Evidence assessment"** gauge - is there an ongoing trial for a specific indication?
* Extract MeSH terms, specified as **mesh_type** to differentiate between direct association vs. hierarchical traversals
    * mesh-list is the extracted concept 
    * mesh-ancestor is an ancestor of that concept

### 1a. Connect to Database

In [2]:
dsn_AACT = ("host='aact-db.ctti-clinicaltrials.org' port='5432' dbname='aact' user='khs2138' password='Max0730!'")
dbcon_AACT = psycopg2.connect(dsn_AACT)
dbcon_AACT.cursor_factory = psycopg2.extras.DictCursor
dbcon_AACT

<connection object at 0x7fac724d2eb0; dsn: 'user=khs2138 password=xxx dbname=aact host=aact-db.ctti-clinicaltrials.org port=5432', closed: 0>

## 1b. Interventions + Conditions: SQL Extract
* STUDIES: 
    * NCT_ID
* CONDITIONS:
    * Condition name
* INTERVENTIONS: 
    * Name - Drug of choice - filter by gabapentin
    * Intervention_type - ensure drug treatment
* BROWSE_INTERVENTIONS:
    * MeSH terms for internvetions
    * MeSH_type for interventions
*  BROWSE_CONDITIONS:
    * MeSH terms for conditions
    * MeSH_type for conditions

In [3]:
sql = """\
SELECT 
	h.*,
    i.mesh_term as cond_mesh,
    i.downcase_mesh_term as d_cond_mesh,
    i.mesh_type 
FROM
	(SELECT 
    	f.*,
        g.mesh_term as drug_mesh,
        g.downcase_mesh_term as d_drug_mesh 
FROM
	(SELECT 
    	e.nct_id,
        e.intervention_name,
        e.condition_name,
        e.intervention_type 
FROM
	(SELECT 
    	c.*,
        d.name as condition_name,
        d.downcase_name 
FROM
	(SELECT 
    	a.nct_id,
        a.name as intervention_name,
        a.intervention_type,
        b.overall_status,
        b.why_stopped,
        b.phase,
        b.brief_title 
FROM
	interventions a
LEFT OUTER JOIN
	studies b
ON
	a.nct_id = b.nct_id
WHERE
	a.name like '%gabapentin%' or a.name like '%Gabapentin%') c
LEFT OUTER JOIN    
	conditions d
ON 
	c.nct_id = d.nct_id) e) f
LEFT OUTER JOIN
	browse_interventions g
ON
	f.nct_id = g.nct_id 
WHERE g.downcase_mesh_term like 'gabapentin') h
LEFT OUTER JOIN
	browse_conditions i
ON 
	h.nct_id = i.nct_id;
"""
logging.debug(f"SQL: {sql}")
aact_sql_extract = read_sql_query(sql, dbcon_AACT)
aact_sql_extract

,nct_id,intervention_name,condition_name,intervention_type,drug_mesh,d_drug_mesh,cond_mesh,d_cond_mesh,mesh_type
0,NCT04860141,Gabapentin 600mg,Dental Pain and Sensation Disorder,Drug,Gabapentin,gabapentin,Sensation Disorders,sensation disorders,mesh-list
1,NCT04860141,Gabapentin 600mg,Dental Pain and Sensation Disorder,Drug,Gabapentin,gabapentin,"Pain, Postoperative","pain, postoperative",mesh-list
2,NCT04860141,Gabapentin 600mg,Dental Pain and Sensation Disorder,Drug,Gabapentin,gabapentin,Postoperative Complications,postoperative complications,mesh-ancestor
3,NCT04860141,Gabapentin 600mg,Dental Pain and Sensation Disorder,Drug,Gabapentin,gabapentin,Pathologic Processes,pathologic processes,mesh-ancestor
4,NCT04860141,Gabapentin 600mg,Dental Pain and Sensation Disorder,Drug,Gabapentin,gabapentin,Pain,pain,mesh-ancestor
...,...,...,...,...,...,...,...,...,...
6401,NCT05024825,Gabapentin,"Pain, Postoperative",Drug,Gabapentin,gabapentin,"Pain, Postoperative","pain, postoperative",mesh-list
6402,NCT05024825,Gabapentin,"Pain, Postoperative",Drug,Gabapentin,gabapentin,Postoperative Complications,postoperative complications,mesh-ancestor
6403,NCT05024825,Gabapentin,"Pain, Postoperative",Drug,Gabapentin,gabapentin,Pathologic Processes,pathologic processes,mesh-ancestor
6404,NCT05024825,Gabapentin,"Pain, Postoperative",Drug,Gabapentin,gabapentin,Pain,pain,mesh-ancestor


## 1c. Extract all treat-relationships from Clinical Trial data for drug in question

In [4]:
# Pull out the necessary columns: 
## NCT_ID: for trial reference
## drug_mesh: for the drug's mesh name 
#### (this should be the same throughout the table since we're pulling above for only ONE drug in question)
## cond_mesh: for the condition mesh terms
## mesh_type: for whether that is directly extracted from clinical trial data OR is an ancestor of the term extracted

# drop_duplicates() - we only want unique 

AACT_cond_mesh = aact_sql_extract[['nct_id','drug_mesh','cond_mesh','mesh_type']].drop_duplicates()
AACT_cond_mesh

,nct_id,drug_mesh,cond_mesh,mesh_type
0,NCT04860141,Gabapentin,Sensation Disorders,mesh-list
1,NCT04860141,Gabapentin,"Pain, Postoperative",mesh-list
2,NCT04860141,Gabapentin,Postoperative Complications,mesh-ancestor
3,NCT04860141,Gabapentin,Pathologic Processes,mesh-ancestor
4,NCT04860141,Gabapentin,Pain,mesh-ancestor
...,...,...,...,...
6401,NCT05024825,Gabapentin,"Pain, Postoperative",mesh-list
6402,NCT05024825,Gabapentin,Postoperative Complications,mesh-ancestor
6403,NCT05024825,Gabapentin,Pathologic Processes,mesh-ancestor
6404,NCT05024825,Gabapentin,Pain,mesh-ancestor


In [5]:
# Reorganize so that we only have each condition once, and its associated mesh types, and a list of all trials
# Rework this: we can priortize mesh_list 
# If something is a mesh_list for any trial, it is a possible direct treat relationship
# If something is a mesh_ancestor and never occurs as a mesh_list, it is only for HIERARCHICAL TRAVERSAL purposes
# so we don't need to know when things map to both 
AACT_conditions = AACT_cond_mesh.groupby(['cond_mesh','mesh_type'])['nct_id'].apply(list)
AACT_conditions

cond_mesh              mesh_type    
Abortion, Spontaneous  mesh-list                                            [NCT02725710]
Acute Pain             mesh-list        [NCT02926573, NCT02703259, NCT02957097, NCT028...
Adenocarcinoma         mesh-ancestor                           [NCT01236053, NCT01138124]
Agnosia                mesh-list                                            [NCT01966172]
Alcohol Drinking       mesh-list                               [NCT03274167, NCT02252536]
                                                              ...                        
Vulvar Diseases        mesh-ancestor                           [NCT00390013, NCT01301001]
Vulvodynia             mesh-list                               [NCT00390013, NCT01301001]
Weight Loss            mesh-list                                            [NCT04613024]
Wounds and Injuries    mesh-ancestor    [NCT04285281, NCT04068506, NCT04025203, NCT038...
                       mesh-list                  [NCT04705480,

In [6]:
AACT_conditions = AACT_conditions.reset_index().sort_values(by='cond_mesh')
AACT_conditions

,cond_mesh,mesh_type,nct_id
0,"Abortion, Spontaneous",mesh-list,[NCT02725710]
1,Acute Pain,mesh-list,"[NCT02926573, NCT02703259, NCT02957097, NCT028..."
2,Adenocarcinoma,mesh-ancestor,"[NCT01236053, NCT01138124]"
3,Agnosia,mesh-list,[NCT01966172]
4,Alcohol Drinking,mesh-list,"[NCT03274167, NCT02252536]"
...,...,...,...
423,Vulvar Diseases,mesh-ancestor,"[NCT00390013, NCT01301001]"
424,Vulvodynia,mesh-list,"[NCT00390013, NCT01301001]"
425,Weight Loss,mesh-list,[NCT04613024]
426,Wounds and Injuries,mesh-ancestor,"[NCT04285281, NCT04068506, NCT04025203, NCT038..."


## 1d. Mapping Conditions from MeSH to SNOMED/ICD/etc.via OMOP vocabulary
CDM Vocabulary reference: https://ohdsi.github.io/CommonDataModel/cdm54.html#Vocabulary_Tables

Downloaded from Athena 
* 05:55 PM | 13-NOV-2021
* V5.0 09-NOV-21


* Currently for ONE MeSh term - need to generalize for all MeSH terms outputted for drug

Takes one MeSH term, identifies the standard OMOP concept for that term, and uses that standard term to then pull all nonStandard terms for that MeSH term. 
* Enables traversal from MeSH --> ICD, SNOMED, etc. 

In [13]:
%cd OMOP_vocabulary

/Users/khs2138/OneDrive - cumc.columbia.edu/Symbolic_Project/OMOP_vocabulary


In [14]:
# Load all concepts
concept = pd.read_csv('CONCEPT.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id','concept_name',
                              'domain_id','vocabulary_id','concept_class_id','standard_concept',
                              'concept_code','valid_start_date','valid_end_date','invalid_reason'],index_col=False)
concept

/Users/khs2138/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,45756805,Pediatric Cardiology,Provider,ABMS,Physician Specialty,S,OMOP4821938,19700101,20991231,NaN
1,45756804,Pediatric Anesthesiology,Provider,ABMS,Physician Specialty,S,OMOP4821939,19700101,20991231,NaN
2,45756803,Pathology-Anatomic / Pathology-Clinical,Provider,ABMS,Physician Specialty,S,OMOP4821940,19700101,20991231,NaN
3,45756802,Pathology - Pediatric,Provider,ABMS,Physician Specialty,S,OMOP4821941,19700101,20991231,NaN
4,45756801,Pathology - Molecular Genetic,Provider,ABMS,Physician Specialty,S,OMOP4821942,19700101,20991231,NaN
...,...,...,...,...,...,...,...,...,...,...
8416980,36118955,Isturisa,Drug,dm+d,Brand Name,NaN,OMOP5047799,19700101,20991231,NaN
8416981,36124032,Supemtek,Drug,dm+d,Brand Name,NaN,OMOP5047800,19700101,20991231,NaN
8416982,36122578,Axpinet,Drug,dm+d,Brand Name,NaN,OMOP5047801,19700101,20991231,NaN
8416983,36122828,Cyclomen,Drug,dm+d,Brand Name,NaN,OMOP5047802,19700101,20991231,NaN


In [15]:
# Load all concept relationships
concept_relationship = pd.read_csv('CONCEPT_RELATIONSHIP.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id_1','concept_id_2',
                              'relationship_id',
                               'valid_start_date','valid_end_date','invalid_reason'],
                           index_col=False)
concept_relationship

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,43141480,43141480,Mapped from,20160807,20991231,NaN
1,43141480,43141480,Maps to,20160807,20991231,NaN
2,43141483,19126920,RxNorm has dose form,20160910,20991231,NaN
3,43141483,43141483,Mapped from,20160807,20991231,NaN
4,43141483,43141483,Maps to,20160807,20991231,NaN
...,...,...,...,...,...,...
54284669,43141472,43174690,Marketed form of,20170624,20991231,NaN
54284670,43141472,43201766,Marketed form of,20170824,20991231,NaN
54284671,43141473,43141473,Mapped from,20160807,20991231,NaN
54284672,43141473,43141473,Maps to,20160807,20991231,NaN


In [12]:
# Identify concept_id for MeSH term 
MeSH_concept = concept.loc[(concept['vocabulary_id']=='MeSH')&(concept['concept_name']==AACT_conditions['cond_mesh'][0])]
MeSH_conceptId = MeSH_concept.reset_index()['concept_id'][0]
display(MeSH_concept)
MeSH_conceptId


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
1719985,45613996,"Abortion, Spontaneous",Condition,MeSH,Main Heading,NaN,D000022,19700101,20991231,NaN


45613996

In [13]:
# Get the entry in the CONCEPT_RELATIONSHIP table for that MeSH term
MeSH_rel_entry = concept_relationship.loc[concept_relationship['concept_id_1']==MeSH_conceptId].reset_index() # NonStandard --> Standard
display(MeSH_rel_entry)

# Get the SNOMED standard ID 
standard_id = MeSH_rel_entry['concept_id_2'][0]
standard_id

,index,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
0,6771407,45613996,4067106,Maps to,20131007,20991231,NaN


4067106

In [14]:
# Get all NonStandard conceptIDs from relationship tables for that SNOMED ID 
NonStandard_Concepts = concept_relationship.loc[(concept_relationship['concept_id_1']==standard_id)&(concept_relationship['relationship_id']=='Mapped from')]  # Get all NonStandard concept IDs
NonStandard_Concepts

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
1535268,4067106,3519999,Mapped from,20200731,20991231,NaN
1535269,4067106,3534301,Mapped from,20200731,20991231,NaN
1535270,4067106,3534302,Mapped from,20200731,20991231,NaN
1535371,4067106,3536059,Mapped from,20200731,20991231,NaN
1535460,4067106,3537760,Mapped from,20200731,20991231,NaN
...,...,...,...,...,...,...
42984233,4067106,45430318,Mapped from,20140401,20991231,NaN
42984234,4067106,45436935,Mapped from,20140401,20991231,NaN
42984235,4067106,45463679,Mapped from,19700101,20991231,NaN
42984236,4067106,45500254,Mapped from,20140401,20991231,NaN


In [15]:
# Join nonstandard conceptIDs with OMOP concept table to get data on the concept name, vocabualry source, standard/non, etc. 
merge_concepts_rels = pd.merge(NonStandard_Concepts, concept,
                        how="left", left_on='concept_id_2', right_on='concept_id')
merge_concepts_rels



,concept_id_1,concept_id_2,relationship_id,valid_start_date_x,valid_end_date_x,invalid_reason_x,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date_y,valid_end_date_y,invalid_reason_y
0,4067106,3519999,Mapped from,20200731,20991231,NaN,3519999,Unspecified abortion with genital tract or pel...,Condition,SNOMED,Clinical Finding,NaN,587061000000104,20100401,20110401,U
1,4067106,3534301,Mapped from,20200731,20991231,NaN,3534301,Unspecified abortion with metabolic disorder,Condition,SNOMED,Clinical Finding,NaN,587071000000106,20100401,20121001,U
2,4067106,3534302,Mapped from,20200731,20991231,NaN,3534302,Unspecified abortion with renal failure,Condition,SNOMED,Clinical Finding,NaN,587081000000108,20100401,20121001,U
3,4067106,3536059,Mapped from,20200731,20991231,NaN,3536059,Unspecified abortion with delayed or excessive...,Condition,SNOMED,Clinical Finding,NaN,614961000000101,20100401,20110401,U
4,4067106,3537760,Mapped from,20200731,20991231,NaN,3537760,Unspecified abortion with shock,Condition,SNOMED,Clinical Finding,NaN,639881000000106,20100401,20121001,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,4067106,45430318,Mapped from,20140401,20991231,NaN,45430318,Unspecified abortion with metabolic disorder,Condition,Read,Read,NaN,L070400,20140401,20991231,NaN
130,4067106,45436935,Mapped from,20140401,20991231,NaN,45436935,Other abnormal product of conception NOS,Condition,Read,Read,NaN,L01z.00,19700101,20991231,NaN
131,4067106,45463679,Mapped from,19700101,20991231,NaN,45463679,Miscarriage,Condition,Read,Read,NaN,L04..11,19700101,20991231,NaN
132,4067106,45500254,Mapped from,20140401,20991231,NaN,45500254,Unspecified spontaneous abortion with shock,Condition,Read,Read,NaN,L040500,20140401,20991231,NaN


In [16]:
# Pull desired columns, rename columns for clarity, and sort by vocabulary for easy reference 
concept_map = merge_concepts_rels[['concept_id_1','concept_id_2','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept','concept_code']]
concept_map =  concept_map.rename(columns = {'concept_id_1':'standard_concept_id','concept_id_':'concept_id'})
concept_map.sort_values(by='vocabulary_id')

,standard_concept_id,concept_id_2,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code
107,4067106,45937965,"Pregnancy, miscarriage",Condition,CIEL,Diagnosis,NaN,48
73,4067106,45927052,Spontaneous abortion,Condition,CIEL,Diagnosis,NaN,126127
42,4067106,45937856,second trimester abortion,Condition,CIEL,Diagnosis,NaN,158735
26,4067106,37611702,Spontaneous abortion,Condition,CIM10,ICD10 Hierarchy,NaN,O03
133,4067106,45592170,Spontaneous abortion,Condition,ICD10,ICD10 Hierarchy,NaN,O03
...,...,...,...,...,...,...,...,...
96,4067106,4113849,Unspecified spontaneous abortion with other sp...,Condition,SNOMED,Clinical Finding,NaN,198640005
61,4067106,4114138,Unspecified abortion with genital tract or pel...,Condition,SNOMED,Clinical Finding,NaN,198770005
76,4067106,3537764,Unspecified abortion NOS,Condition,SNOMED,Clinical Finding,NaN,639931000000103
0,4067106,3519999,Unspecified abortion with genital tract or pel...,Condition,SNOMED,Clinical Finding,NaN,587061000000104


# 2. DrugCentral
DrugCentral is online drug information resource created and maintained by Division of Translational Informatics at University of New Mexico in collaboration with the IDG.


DrugCentral provides information on active ingredients chemical entities, pharmaceutical products, drug mode of action, indications, pharmacologic action. We monitor FDA, EMA, and PMDA for new drug approval on regular basis to ensure currency of the resource. Limited information on discontinued and drugs approved outside US is also available however regulatory approval information can't be verified. Database developed and maintained by Oleg Ursu and Tudor Oprea. Web application developed by Jayme Holmes.

https://drugcentral.org/download

**Utility for project**:
* Data on indication, off-label drug use 
* Active ingredient data
* Contians OMOP data linked with FDA data, ingredient, etc. 

### 2a. Connect to Database


In [3]:
dsn_DC = ("host='unmtid-dbs.net' port='5433' dbname='drugcentral' user='drugman' password='dosage'")
dbcon_DC = psycopg2.connect(dsn_DC)
dbcon_DC.cursor_factory = psycopg2.extras.DictCursor
dbcon_DC

<connection object at 0x7fb9a18d8eb0; dsn: 'user=drugman password=xxx dbname=drugcentral host=unmtid-dbs.net port=5433', closed: 0>

In [4]:
# Check Version
sql=(f"SELECT * FROM public.dbversion")
logging.debug(f"SQL: {sql}")
df = read_sql_query(sql, dbcon_DC)
df

,version,dtime
0,49,2021-09-24 09:24:23.962


## 2b. Extract Indications
Leveraging: 
* Drug Central public.omop
* Drug Central public.structure

In [5]:
# Pull ALL indications from OMOP table, map to drug name using strucutre table from Drug Central
sql = """\
SELECT 
	omop.struct_id, 
    omop.concept_id as omop_concept_id, 
    omop.relationship_name, 
    omop.concept_name as omop_concept_name, 
    omop.umls_cui, 
    omop.cui_semantic_type as umls_semantic_type,
    omop.snomed_conceptid,
    omop.snomed_full_name,
    struct.name as drug_name
from public.omop_relationship omop
JOIN
	public.structures struct 
ON omop.struct_id = struct.id
"""
logging.debug(f"SQL: {sql}")
omop_struct_sql_extract= read_sql_query(sql, dbcon_DC)
omop_struct_sql_extract

,struct_id,omop_concept_id,relationship_name,omop_concept_name,umls_cui,umls_semantic_type,snomed_conceptid,snomed_full_name,drug_name
0,5391,40249429,indication,Triple negative breast neoplasms,C3539878,T191,7.069700e+08,Triple negative breast neoplasms,sacituzumab govitecan
1,5392,40249397,indication,Metastatic non-small cell lung cancer,C0278987,T191,NaN,Metastatic non-small cell lung cancer,capmatinib
2,5393,40249397,indication,Metastatic non-small cell lung cancer,C0278987,T191,NaN,Metastatic non-small cell lung cancer,selpercatinib
3,5393,40248953,indication,Medullary thyroid carcinoma,C0238462,T191,2.550320e+08,Medullary thyroid carcinoma,selpercatinib
4,5394,21004733,indication,Gastrointestinal stromal tumor,C0238198,T191,4.201200e+08,Gastrointestinal stromal tumor,ripretinib
...,...,...,...,...,...,...,...,...,...
41425,1982,21003217,off-label use,Nausea and vomiting,C0027498,T184,1.693200e+07,Nausea and vomiting,olanzapine
41426,1982,40249519,off-label use,cannabinoid hyperememsis,C4511590,T047,1.104788e+16,Cannabis hyperemesis syndrome co-occurrent and...,olanzapine
41427,1982,40249520,off-label use,cannabinoid hyperememsis,C4511594,T047,1.104801e+16,Cannabis hyperemesis syndrome co-occurrent and...,olanzapine
41428,4209,40249521,off-label use,PTSD related nightmares,C1997862,T048,4.286870e+08,Nightmares associated with chronic post-trauma...,prazosin


### 2c. Extract indications for drug of choice

In [6]:
# Extract specifically for gabapentin
omop_indications = omop_struct_sql_extract.loc[omop_struct_sql_extract['drug_name']=='gabapentin'].sort_values(by='relationship_name').reset_index(drop=True)
omop_indications

## Disregard contraindication

,struct_id,omop_concept_id,relationship_name,omop_concept_name,umls_cui,umls_semantic_type,snomed_conceptid,snomed_full_name,drug_name
0,1264,21001022,contraindication,Depressive disorder,C0011581,T048,3.548901e+07,Depressive disorder,gabapentin
1,1264,21000985,contraindication,Suicidal thoughts,C0424000,T033,6.471006e+06,Suicidal thoughts,gabapentin
2,1264,21002292,contraindication,Kidney disease,C0022658,T047,9.070800e+07,Kidney disease,gabapentin
3,1264,21000166,indication,Postherpetic neuralgia,C0032768,T047,2.177002e+06,Postherpetic neuralgia,gabapentin
4,1264,21004851,indication,Restless legs,C0035258,T047,3.291401e+07,Restless legs,gabapentin
5,1264,21013508,indication,Partial Epilepsy Treatment Adjunct,None,None,NaN,None,gabapentin
6,1264,21002508,off-label use,Menopausal flushing,C0600142,T184,1.984360e+08,Menopausal flushing,gabapentin
7,1264,21003090,off-label use,Chronic pain,C0150055,T184,8.242300e+07,Chronic pain,gabapentin
8,1264,40249515,off-label use,Refractory pruritis,C0033774,T184,4.183630e+08,Itching of skin,gabapentin
9,1264,4264789,off-label use,Anxiety disorder,C0003469,T048,1.974800e+08,Anxiety disorder,gabapentin


### Reformat 
1. SNOMED_CONCEPT_ID from drug central is actually the SNOMED_concept_code
* We want to match on concept_ID, to traverse through OMOP
2. Reformat concept_ID column as int

In [18]:
# Convert SNOMED CONCEPT_ID (really: CONCEPT_CODE) to int
omop_indications['snomed_conceptid'] = omop_indications['snomed_conceptid'].fillna(0).astype(np.int64) 

# Join nonstandard conceptIDs with OMOP concept table to get data on the concept name, vocabualry source, standard flag, etc. 
merge_DC = pd.merge(omop_indications, concept, 
                    how="left", left_on='snomed_conceptid', right_on='concept_code')
merge_DC = merge_DC.loc[merge_DC['vocabulary_id']!='Nebraska Lexicon'].reset_index()
merge_DC

DC_indications = merge_DC[['struct_id','omop_concept_id','omop_concept_name','relationship_name','umls_cui','umls_semantic_type','concept_id','snomed_conceptid','snomed_full_name','drug_name','concept_class_id','standard_concept']]
DC_indications =  DC_indications.rename(columns = {'snomed_conceptid':'snomed_concept_code','concept_id':'snomed_concept_id'})
DC_indications['snomed_concept_id'] = DC_indications['snomed_concept_id'].fillna(0).astype(np.int64)
DC_indications

,struct_id,omop_concept_id,omop_concept_name,relationship_name,umls_cui,umls_semantic_type,snomed_concept_id,snomed_concept_code,snomed_full_name,drug_name,concept_class_id,standard_concept
0,1264,21001022,Depressive disorder,contraindication,C0011581,T048,440383,35489007,Depressive disorder,gabapentin,Clinical Finding,S
1,1264,21000985,Suicidal thoughts,contraindication,C0424000,T033,4273391,6471006,Suicidal thoughts,gabapentin,Clinical Finding,S
2,1264,21002292,Kidney disease,contraindication,C0022658,T047,198124,90708001,Kidney disease,gabapentin,Clinical Finding,S
3,1264,21000166,Postherpetic neuralgia,indication,C0032768,T047,4071164,2177002,Postherpetic neuralgia,gabapentin,Clinical Finding,S
4,1264,21004851,Restless legs,indication,C0035258,T047,73754,32914008,Restless legs,gabapentin,Clinical Finding,S
5,1264,21013508,Partial Epilepsy Treatment Adjunct,indication,None,None,0,0,None,gabapentin,NaN,NaN
6,1264,21002508,Menopausal flushing,off-label use,C0600142,T184,4113666,198436008,Menopausal flushing,gabapentin,Clinical Finding,S
7,1264,21003090,Chronic pain,off-label use,C0150055,T184,436096,82423001,Chronic pain,gabapentin,Clinical Finding,S
8,1264,40249515,Refractory pruritis,off-label use,C0033774,T184,4169287,418363000,Itching of skin,gabapentin,Clinical Finding,S
9,1264,4264789,Anxiety disorder,off-label use,C0003469,T048,442077,197480006,Anxiety disorder,gabapentin,Clinical Finding,S


In [19]:
DC_indications.to_csv('Gabapentin_indications_drugcentral.csv')

## 2d. Mapping Conditions from OMOP/SNOMED to ICD/etc. via OMOP vocabulary

In [21]:
# Get the SNOMED standard name 
standard_name_DC = omop_indications['snomed_full_name'][0]
standard_name_DC

'Depressive disorder'

In [22]:
# Get the SNOMED standard concept_ID from the concept_name using concept table
standard_id_DC = concept.loc[(concept['concept_name']==standard_name_DC)&(concept['standard_concept']=='S')]['concept_id'].reset_index(drop=True)
standard_id_DC = standard_id_DC[0]
standard_id_DC


440383

In [23]:
# Get all NonStandard conceptIDs from relationship tables for that SNOMED ID 
NonStandard_Concepts_DC = concept_relationship.loc[(concept_relationship['concept_id_1']==standard_id_DC)&(concept_relationship['relationship_id']=='Mapped from')]  # Get all NonStandard concept IDs
NonStandard_Concepts_DC

,concept_id_1,concept_id_2,relationship_id,valid_start_date,valid_end_date,invalid_reason
14561603,440383,3528972,Mapped from,20200731,20991231,NaN
14664354,440383,1410661,Mapped from,20160101,20991231,NaN
14664355,440383,1410662,Mapped from,20160101,20991231,NaN
14664356,440383,3122804,Mapped from,19700101,20991231,NaN
14664357,440383,3132802,Mapped from,19700101,20991231,NaN
...,...,...,...,...,...,...
51198864,440383,45446561,Mapped from,20150917,20991231,NaN
51198865,440383,45463353,Mapped from,20140401,20991231,NaN
51198866,440383,45473438,Mapped from,20150917,20991231,NaN
51198867,440383,45523245,Mapped from,20150917,20991231,NaN


In [24]:
# Join nonstandard conceptIDs with OMOP concept table to get data on the concept name, vocabualry source, standard/non, etc. 
merge_concepts_rels_DC = pd.merge(NonStandard_Concepts_DC, concept,
                        how="left", left_on='concept_id_2', right_on='concept_id')
merge_concepts_rels_DC


,concept_id_1,concept_id_2,relationship_id,valid_start_date_x,valid_end_date_x,invalid_reason_x,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date_y,valid_end_date_y,invalid_reason_y
0,440383,3528972,Mapped from,20200731,20991231,NaN,3528972,[X] Depression: [other episodes] or [atypical]...,Condition,SNOMED,Clinical Finding,NaN,419311000000100,19700101,20090401,U
1,440383,1410661,Mapped from,20160101,20991231,NaN,1410661,"Depressive episode, unspecified",Condition,ICD10CN,ICD10 code,NaN,F32.900,20160101,20991231,NaN
2,440383,1410662,Mapped from,20160101,20991231,NaN,1410662,Depression (machine translation),Condition,ICD10CN,ICD10 code,NaN,F32.901,20160101,20991231,NaN
3,440383,3122804,Mapped from,19700101,20991231,NaN,3122804,Depressive disorder NEC,Condition,Nebraska Lexicon,Clinical Finding,NaN,192078003,20020130,20020131,U
4,440383,3132802,Mapped from,19700101,20991231,NaN,3132802,Mood disorder with depressive feature,Condition,Nebraska Lexicon,Clinical Finding,NaN,21061000119107,20170131,20170731,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,440383,45446561,Mapped from,20150917,20991231,NaN,45446561,[X]Depressive episode,Condition,Read,Read,NaN,Eu32.00,19700101,20991231,NaN
91,440383,45463353,Mapped from,20140401,20991231,NaN,45463353,[X]Monopolar depression NOS,Condition,Read,Read,NaN,Eu33z11,19700101,20991231,NaN
92,440383,45473438,Mapped from,20150917,20991231,NaN,45473438,[X]Depressive disorder NOS,Condition,Read,Read,NaN,Eu32z12,19700101,20991231,NaN
93,440383,45523245,Mapped from,20150917,20991231,NaN,45523245,[X]Other depressive episodes,Condition,Read,Read,NaN,Eu32y00,19700101,20991231,NaN


In [25]:
# Pull desired columns, rename columns for clarity, and sort by vocabulary for easy reference 
concept_map_DC = merge_concepts_rels_DC[['concept_id_1','concept_id_2','concept_name','domain_id','vocabulary_id','concept_class_id','standard_concept','concept_code']]
concept_map_DC =  concept_map_DC.rename(columns = {'concept_id_1':'standard_concept_id','concept_id_2':'concept_id'}).sort_values(by='vocabulary_id')
pd.set_option('display.max_rows', concept_map_DC.shape[0]+1)
concept_map_DC


,standard_concept_id,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code
47,440383,45919614,DEPRESSIVE EPISODE,Condition,CIEL,Diagnosis,NaN,207
27,440383,45913939,Melancholia,Condition,CIEL,Diagnosis,NaN,119535
28,440383,45920960,Depressive Disorder,Condition,CIEL,Diagnosis,NaN,142563
29,440383,45922522,DEPRESSIVE EPISODE,Condition,CIEL,Diagnosis,NaN,911
65,440383,45908377,Depression,Condition,CIEL,Diagnosis,NaN,119537
77,440383,37604841,subcortical vascular dementia with other sympt...,Condition,CIM10,ICD10 code,NaN,F01.230
76,440383,37604779,"Dementia disease due to HIV [HIV], with other ...",Condition,CIM10,ICD10 code,NaN,F02.431
75,440383,37604748,"Dementia in Huntington's disease, with other s...",Condition,CIM10,ICD10 code,NaN,F02.231
74,440383,37604747,"Dementia in Huntington's disease, with other s...",Condition,CIM10,ICD10 code,NaN,F02.230
73,440383,37604686,"Dementia in Pick's disease, with other symptom...",Condition,CIM10,ICD10 code,NaN,F02.03
